# Pre-processing UERRA and EURO-CORDEX datasets

Visit the [how to run risk workflows](https://handbook.climaax.eu/notebooks/workflows_how_to.html) page in the CLIMAAX Handbook and the [GitHub repository](https://github.com/CLIMAAX/HAZARD) of this workflow for information on how to use this Jupyter notebook.

## Download datasets
You can download the dataset UERRA regional reanalysis in the following website: https://cds.climate.copernicus.eu/datasets/reanalysis-uerra-europe-single-levels?tab=overview.
The UERRA dataset contains analyses of surface and near-surface essential climate variables from UERRA-HARMONIE and MESCAN-SURFEX systems. For this assessment the MESCAN-SURFEC was download considering two different variables:
- Air Temperature: Measured at 2 meters above the surface (commonly referred to as 2m temperature).
- Total Precipitation: The total amount of water (both liquid and solid forms) falling onto the ground or water surface. This dataset includes all types of precipitation and represents an accumulated value over 24 hours, from 06:00 on one day to 06:00 the following day.

On the cell below, an example for the download of 2 meter tempeture for year 1981

In [ ]:
import cdsapi

dataset = "reanalysis-uerra-europe-single-levels"
request = {
    "origin": "mescan_surfex",
    "variable": "2m_temperature",
    "year": ["1981"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00", "06:00", "12:00",
        "18:00"
    ],
    "data_format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

## Process the data
This code processes multiple NetCDF files from the input folder (data). It automates several key tasks, including converting temperature data from Kelvin to Celsius and adding the units for both temperature (°C) and precipitation (mm/day). Additionally, the code clips the datasets to a specified time period (1981-2010) and saves the modified files in a specified output directory (data_conv).

In [ ]:
import os
import xarray as xr

In [ ]:
def process_netcdf_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f'Created output folder: {output_folder}')

    # Loop through all the files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.nc'):
            input_file_path = os.path.join(input_folder, filename)
            output_file_path = os.path.join(output_folder, filename)
            print(f'Processing file: {input_file_path}')

            # Load and clip the dataset
            ds = xr.open_dataset(input_file_path)
            ds_subset = ds.sel(time=slice('1981', '2010'))

            # Convert temperature from Kelvin to Celsius
            if 't2m' in ds_subset:
                ds_subset['t2m'] = ds_subset['t2m'] - 273.15
                ds_subset['t2m'].attrs['units'] = 'C'  # Add units attribute
                print(f'Converted t2m to Celsius')

            # Convert precipitation from kg/m²= mm/day
            if 'tp' in ds_subset:
                ds_subset['tp'] = ds_subset['tp']
                ds_subset['tp'].attrs['units'] = 'mm/d'  # Add units attribute
                print(f'Converted pr to mm/day')

            # Save the processed and clipped data to a new NetCDF file
            ds_subset.to_netcdf(output_file_path)
            print(f'Saved processed file to: {output_file_path}')

## Save dataset to a local directory for future access
We will save the processed files to a local folder to be able to easily access it later.

In [ ]:
# Define the input, output, and shapefile paths
input_folder = "../data"  # UERRA datasets are located
output_folder = "../data_conv"  # data saved after unit format conversion

process_netcdf_files(input_folder, output_folder)